# Multi-agent Systems

Multi-agent systems solve two fundamental problems that emerge when building sophisticated LLM applications: task complexity and context window constraints.

When you build a single monolithic agent to handle complex workflows, you quickly hit limitations. The agent's prompt must contain instructions for all possible scenarios, tool definitions for every action it might take, and examples covering diverse use cases. LLMs perform best when given focused, specific instructions. A 5,000-token system prompt trying to cover everything will naturally produce less accurate results than a 500-token prompt focused on one domain.

Furthermore, it's difficult to debug a complex agent. When something goes wrong, it's hard to isolate whether the issue is in the routing logic, domain knowledge, or tool execution. Everything is entangled.

A multi-agent approach decomposes this into specialized components, each with a narrow focus and minimal context requirements.

In [ ]:
from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.messages import HumanMessage
from langchain.tools import tool
from loguru import logger
from pydantic import BaseModel, Field

from chain_reaction.config import APIKeys, ModelBehavior, ModelName
from chain_reaction.utils import get_last_message, get_structured_response

# Define chat model
api_keys = APIKeys()

chat_model = init_chat_model(
    model=ModelName.CLAUDE_HAIKU,
    timeout=None,
    max_retries=2,
    api_key=api_keys.anthropic,
    **ModelBehavior.deterministic().model_dump(),
)

# Set logger level
logger.level("DEBUG")

# Define sub-agents

In [ ]:
class CalculationResult(BaseModel):
    """Response model for calculation results."""

    result: float = Field(description="The result of the calculation.")

    @classmethod
    def get_response(cls, response: dict) -> CalculationResult | None:
        """Extract a CalculationResult instance from the agent response.

        Args:
            response (dict): The agent response containing the structured response.

        Returns:
            CalculationResult | None: The extracted CalculationResult if available, otherwise None.
        """
        return get_structured_response(response, cls)


@tool
def calculate_square_root(number: float) -> float:
    """Calculates the square root of a given number.

    Args:
        number (float): The number to calculate the square root of.

    Returns:
        float: The square root of the number.
    """
    return number**0.5


@tool
def calculate_square(number: float) -> float:
    """Calculates the square of a given number.

    Args:
        number (float): The number to calculate the square of.

    Returns:
        float: The square of the number.
    """
    return number**2


square_root_agent = create_agent(
    model=chat_model,
    tools=[calculate_square_root],
    system_prompt="""
    You're a helpful assistant that can perform square root calculations.
    Use the provided calculation tools to answer user questions accurately.
    """,
    response_format=CalculationResult,
)

square_agent = create_agent(
    model=chat_model,
    tools=[calculate_square],
    system_prompt="""
    You're a helpful assistant that can perform square calculations.
    Use the provided calculation tools to answer user questions accurately.
    """,
    response_format=CalculationResult,
)

In [ ]:
response = square_root_agent.invoke({"messages": [HumanMessage(content="What is the square root of 16?")]})
CalculationResult.get_response(response)

# Define main agent

In [ ]:
@tool
def invoke_square_agent(number: float) -> CalculationResult:
    """Invokes the square agent to calculate the square of a number.

    Args:
        number (float): The number to calculate the square of.

    Returns:
        CalculationResult: The result of the square calculation.
    """
    logger.debug(f"Invoking square agent for number: {number}")
    response = square_agent.invoke({"messages": [HumanMessage(content=f"What is the square of {number}?")]})
    result = CalculationResult.get_response(response)
    if result is not None:
        return result
    else:
        raise ValueError("Failed to get square calculation result.")


@tool
def invoke_square_root_agent(number: float) -> CalculationResult:
    """Invokes the square root agent to calculate the square root of a number.

    Args:
        number (float): The number to calculate the square root of.

    Returns:
        CalculationResult: The result of the square root calculation.
    """
    logger.debug(f"Invoking square root agent for number: {number}")
    response = square_root_agent.invoke({"messages": [HumanMessage(content=f"What is the square root of {number}?")]})
    result = CalculationResult.get_response(response)
    if result is not None:
        return result
    else:
        raise ValueError("Failed to get square calculation result.")


main_agent = create_agent(
    model=chat_model,
    tools=[invoke_square_root_agent, invoke_square_agent],
    system_prompt="""
    You're a helpful assistant that can perform both square and square root calculations.
    Use the provided sub-agents to answer user questions accurately.
    """,
)

In [ ]:
response = main_agent.invoke({"messages": [HumanMessage(content="What is the square of the square root of 81?")]})
last_message = get_last_message(response)
if last_message:
    print(last_message.content)